# What is JuMP?

JuMP is an _modeling language_ for optimization problems, writen in julia. 

When solving an optimization problem, typically you start with something like this:

![alt text](img/pooling_problem.pdf)

To solve this, you might use a _solver_: a software implementation of an optimization algorithm. They typically want the problem specified in a much more opaque way:

```java
import ilog.concert.*;
import ilog.cplex.*;
public class Example {
 public static void main(String[] args) {
   try {
    IloCplex cplex = new IloCplex();
double[] lb = {0.0, 0.0, 0.0};
double[] ub = {40.0, Double.MAX_VALUE, Double.MAX_VALUE}; IloNumVar[] x = cplex.numVarArray(3, lb, ub);
    double[] objvals = {1.0, 2.0, 3.0};
6
cplex.addMaximize(cplex.scalProd(x, objvals));
    cplex.addLe(cplex.sum(cplex.prod(-1.0, x[0]),
                   cplex.prod( 1.0, x[1]),
                   cplex.prod( 1.0, x[2])), 20.0);
    cplex.addLe(cplex.sum(cplex.prod( 1.0, x[0]),
                   cplex.prod(-3.0, x[1]),
                   cplex.prod( 1.0, x[2])), 30.0);
if ( cplex.solve() ) {
cplex.out().println("Solution status = " + cplex.getStatus()); cplex.out().println("Solution value = " + cplex.getObjValue());
     double[] val = cplex.getValues(x);
     int ncols = cplex.getNcols();
     for (int j = 0; j < ncols; ++j)
       cplex.out().println("Column: " + j + " Value = " + val[j]);
    }
    cplex.end();
   }
   catch (IloException e) {
    System.err.println("Concert exception '" + e + "' caught");
} }
}
```

For larger, more complex problem, programming like this is:
* Time-consuming
* Difficult
* Hard to maintain/extend
* Error-prone

A modeling language (like JuMP) let's you code an optimization problem in a more natural way. It does the translation to the low-level solver format for you.

There are a number of modeling languages out there. Why JuMP?

* User-friendly
* Matches performance of competitors
* Solver-independent
* Easy to extend and take advantage of advanced features

In this session, we will focus on using JuMP for linear optimization. In the next class, we'll see how you can use if for discrete (i.e. integer) optimization, and for general nonlinear optimization.

# Installing JuMP

To install JuMP, just run

In [ ]:
Pkg.add("JuMP")

We already did this in the preassignment. To actually solve a problem, we will also need to install a solver package. There are 15+ options exposed in julia, each with support for different problem classes, different performance profiles, licensing requirements, etc. For the preassignment, we installed Gurobi, a best-of-breed linear/integer programming solver with a generous academic license.

In [ ]:
Pkg.add("Gurobi")

# A first example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First, we load the JuMP and Gurobi libraries.

In [ ]:
using JuMP, Gurobi

Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.

In [ ]:
model = Model(solver=GurobiSolver())

Next, we define the two decision variables in our optimization problem. We will use the ``@variable`` macro (a fancy function, essentially). The first argument is the model object to attach the variable to, and the second specifies the variable name and any bounds.

In [ ]:
@variable(model, x >= 0)
@variable(model, y >= 0)

We now add the single constraint of our problem using the ``@constraint`` macro. We write it algebraically, exactly as we see it above.

In [ ]:
@constraint(model, x + y <= 1)

We specify the objective function with the ``@objective`` macro.

In [ ]:
@objective(model, Max, x + 2y)

In [ ]:
model

To solve the optimization problem, call the ``solve`` function.

In [ ]:
solve(model)

We can now inspect the solution values and optimal cost.

In [ ]:
@show getvalue(x)
@show getvalue(y)
@show getobjectivevalue(model)

# Exercise

Code and solve the following optimization problem:

$$
\begin{align*}
\min_{x,y} \quad& 3x - y \\
\text{s.t.}\quad& x + 2y \geq 1 \\
& x \geq 0 \\
& 0 \leq y \leq 1.
\end{align*}
$$

# Airline Network Revenue Management

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

In the airline network revenue management problem we are trying to decide how many tickets for each origin-destination (O-D) pair to sell at each price level. The goal is to maximize revenue, and we cannot sell more tickets than there is demand for, or space on the planes for.

## Three Flight Problem

We'll start with a toy problem that has three origin-destination pairs, with two price classes for each pair. The three origin-destination pairs are BOS-MDW, MDW-SFO, or BOS-SFO via MDW. BOS stands for Boston, MDW is Chicago-Midway, and SFO is San Francisco. Each O-D pair has a "regular" and "discount" fare class. The data we will use is summarized as follows:

```
PLANE CAPACITY: 166

BOS-MDW
        Regular  Discount
Price:  428      190
Demand: 80       120

BOS-SFO
        Regular  Discount
Price:  642      224
Demand: 75       100

MDW-SFO
        Regular  Discount
Price:  512      190
Demand: 60       110
```

In [ ]:
nrm = Model(solver=GurobiSolver())

In [ ]:
@variables(nrm, begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end)
nrm

In [ ]:
@objective(nrm, Max, 428BOStoMDW_R + 190BOStoMDW_D +
                     642BOStoSFO_R + 224BOStoSFO_D +
                     512MDWtoSFO_R + 190MDWtoSFO_D)
nrm

In [ ]:
@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
nrm

In [ ]:
status = solve(nrm)
status  # Should be `:Optimal`

In [ ]:
@show getvalue(BOStoMDW_R)
@show getvalue(BOStoMDW_D)
@show getobjectivevalue(nrm)

At this point, an exercise might be to extend this model by, say, adding another airport in this same fashion. I won't assign that, though, because it's a little tedious. It also doesn't scale well to problems with many airports. Instead, we can use JuMP's collections and summation notation to make compact, clear, and maintainable models for larger, more complex problems.

First, we would like to construct a _collection of variables_ all at once.  This is a very common idiom; for example, you might have a variable named ``x`` that is indexed from 1 to 10:

In [ ]:
m = Model()
@variable(m, x[1:10] >= 0)

The index sets are specified inside the ``[...]`` block. You can create multidimensional containers by specifying multiple index sets, separated by commas:

In [ ]:
@variable(m, y[1:10,["red","blue"]] <= 1)

For more complicated expressions, you can name the indices for the index sets and use them in the rest of the variable definition:

In [ ]:
ub = rand(10)
@variable(m, i <= z[i=1:10,j=(i+1):10] <= ub[j])

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [ ]:
@variable(m, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Now that we can programatically create arrays of variables, we would like to be able to use them to full-effect in the constraints of our problem. That is, we want a way to express multi-dimensional summations, with conditionals. To do this, we use the ``sum(...)`` construction. The first argument is the ''inner loop'' of the summation, the index sets are specified after a ``for``, and any conditionals are stated following an ``if`` (similar to variable definition, but with a slightly different syntax).

In [ ]:
@constraint(m, sum(x[i] for i in 1:10) <= 1)

In [ ]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(m, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

In [ ]:
@constraint(m, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

Now let's return to the network revenue management example and attempt to rewrite it in a generic way that scales to any number of airports. 

First, let's create some random data for our problem.

In [ ]:
# Set the random seed to ensure we always
# get the same stream of 'random' numbers
srand(1988)  

# Lets create a vector of symbols, one for each airport
airports = [:BOS, :MDW, :SFO, :YYZ]
num_airport = length(airports)

# We'll also create a vector of fare classes
classes = [:REG, :DIS]

# All the demand and price data for each triple of
# (origin, destination, class) will be stored in
# 'dictionaries', also known as 'maps'.
demand = Dict()
prices = Dict()

# Generate a demand and price for each pair of airports
# To keep the code simple we will generate info for
# nonsense flights like BOS-BOS and SFO-SFO, but they
# won't appear in our final model.
for origin in airports, dest in airports
    # Generate demand:
    #  - Regular demand is Uniform(50,90)
    #  - Discount demand is Uniform(100,130)
    demand[(origin,dest,:REG)] = rand(50:90)    
    demand[(origin,dest,:DIS)] = rand(100:130)
    # Generate prices:
    #  - Regular price is Uniform(400,700)
    #  - Discount price is Uniform(150,300)
    prices[(origin,dest,:REG)] = rand(400:700)
    prices[(origin,dest,:DIS)] = rand(150:300)
end

# Finally set all places to have the same capacity
plane_cap = rand(150:200)

# Lets look at a sample demand at random
@show demand[(:BOS,:YYZ,:REG)]

Now let's build the model. We will have our decision variable ``x`` indexed by three things:

1. Origin
2. Destination
3. Class

The upper bound (the demand for each) will vary accordingly.

In [ ]:
nrm2 = Model()

@variable(nrm2, 0 <= x[o=airports,
                       d=airports,
                       c=classes; o!=d] <= demand[(o,d,c)])
nrm2

The objective is to maximize the profit we make, summing over each ticket set:

In [ ]:
@objective(nrm2, Max, sum(prices[(o,d,c)]*x[o,d,c] for 
    o in airports, d in airports, c in classes if o != d))
nrm2

Our first set of constraints enforces that all the legs leaving the hub airport must not oversell the plane capacity:

In [ ]:
for d in airports
    if d != :MDW
        println("Adding constraint for hub (MDW) to $d")
        @constraint(nrm2, 
            sum(x[o,d,c] for o in airports, c in classes if o!=d) <= plane_cap)
    end
end
nrm2

Now, as an exercise, complete this model by adding constraints that each flight _to_ the hub is not oversold.

In [ ]:
# Constraints here!
             
# Now solve the model
solve(nrm2)
@show getvalue(x)
@show getobjectivevalue(nrm2)

# Exercise: Chebyshev Center

Take some polyhedron $P = \{x \in \mathbb{R}^n : Ax \leq b\}$. Now take the largest ball
$$
B = \{x + u : ||u||_2 \leq r\}
$$
given by center $x$ and radius $r$ such that $B \subset P$. The center $x$ is the Chebyshev center of the polyhedron.

We can find the Chebyshev center using linear optimization. We enforce that the ball $B$ lies inside the polyhedron $P$ by requiring that, for each constraint $i$ ($=1\ldots,m$),
$$
A_i(x+u) \leq b_i \quad \forall u : ||u||_2 \leq r.
$$
Since $\sup_{u : ||u||_2 \leq r} A_i x = r||A_i||_2$, the constraint is equivalent to
$$
A_ix + r ||A_i||_2 \leq b_i.
$$
Therefore, the problem of finding the Chebyshev center is equivalent to
$$
\begin{align*}
\max_{x,r} \quad& r \\
\text{s.t.} \quad& A_i x + ||A_i||_2r \leq b_i \quad \forall i = 1,\ldots,m.
\end{align*}
$$
Your exercise is to code up this optimization problem and solve it, using the problem data I give you.

In [ ]:
n = 2
m = 4
# Store LHS as vector-of-vectors
A = Vector{Float64}[
    [ 2, 1], [ 2,-1],
    [-1, 2], [-1,-2]]
b = ones(m)

# Build JuMP model
model = Model(solver=GurobiSolver())

# Your model goes here!
# HINT: The dot(x,y) function will be useful for writing the constraints
@variable(model, r)
@variable(model, x[1:2])
@objective(model, Max, r)
for i in 1:m
    @constraint(model, dot(A[i],x) + norm(A[i])*r <= b[i])
end

# Now solve the model
solve(model)
@show getvalue(x)
@show getvalue(r);

If you solved the problem above correctly, you can visualize the solution using the code below.

In [ ]:
# Use Gadfly to display the solution
Pkg.add("Gadfly")

using Gadfly
Gadfly.set_default_plot_size(8cm, 8cm)
# Plot lines over [-1.5, 1.5]
domain = linspace(-1, +1)
# Plot circle across all angles
θ = linspace(0,2π)
plot(
# a_1 x_1 + a_2 x_2 = b
# --> x_2 = (b - a_1 x_1)/a_2
[layer(x=domain,
       y=(b[i]-A[i][1]*domain)/A[i][2],
       Geom.line,
       Theme(line_width=2px,
             default_color=colorant"blue")) for i in 1:4]...,
# x_1' = x_1 + rθ
# x_2' = x_2 + rθ
layer(x=getvalue(x[1]) + getvalue(r)*cos(θ),
      y=getvalue(x[1]) + getvalue(r)*sin(θ),
      Geom.path,
      Theme(line_width=5px,
            default_color=colorant"red")),
Coord.Cartesian(ymin=-1,ymax=+1)
)